<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Pauli's, and Clifford's

This document defines data structures and standard subroutines for Pauli operators, Clifford operators, and stabilizer states.

In [1]:
import numpy as np
from qiskit.quantum_info import Pauli, pauli_group

# Notation 

We consider bit strings as vectors in a $n$-dimensional finite field GF(2). Given two vectors $u,v \in \{0,1\}^n$ let $u\cdot v =  \sum_{j=0}^{n-1} u_jv_j$. We will use the notation $\oplus$ for the addition of binary vectors modulo two. Define basis vectors

$$e^0 = 100...0$$
$$e^1 = 010...0$$ 
$$e^2 = 001...0$$ 
$$...$$
$$e^n = 000...1$$

If $M$ is a binary matrix with n rows, then the $j$-th row of $M$ can be retrieved by the product $e^j M$. 

## Pauli Group

Given a $n$- dimensional binary vector $v$ and a single-qubit operator $P$ let $P(v) = P^{v_{n-1}} \otimes···\otimes P^{v_0}$ be the tensor product of $P$ single-qubit operators at locations where $v_j=1$. We define then $P_j$ as the single-qubit operator $P$ acting on qubit $j$, that is, $P_j  \equiv  P(e^j)$. 
The single-qubit Pauli operators are denoted $X$, $Y$, $Z$, and represented by 2x2 matrices acting on the state of one qubit,

$$X = \begin{pmatrix} 0 & 1  \\  1 & 0\\\end{pmatrix} ~~ Y = \begin{pmatrix} 0 & -i  \\  i & 0\\\end{pmatrix} ~~ Z = \begin{pmatrix} 1 & 0  \\  0 & -1\\\end{pmatrix}$$

Before introducing our way of representing Pauli operators in Qiskit, let us first check here that the matrices defined above verify the well-known equivalence $ ZX = iY$

In [5]:
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

print(np.dot(Z,X)==1j*Y)

[[ True  True]
 [ True  True]]


In Qiskit Terra we represent a Pauli operator $P_{zx}$ acting on n qubits via a pair of binary vectors $z, x \in \{0, 1\}^n$ such that

$$\begin{equation} P_{zx} = (-i)^{z\cdot x}  Z^z X^x\end{equation} $$

where z and x are elements of $Z_2^n$. That is, there are $4^n$ elements (no phases in this group). The term $(-i)^{z\cdot x}$ essentially multiplies by a factor $-i$ for each qubit index where both the bitstrings of $z$ and $x$ are 1.

Following this definition, multiplication between two Pauli operators can be obtained as

$$PP′ =(-i)^{(z\oplus z′)\cdot (x\oplus x′)} Z^{(z\oplus z′)}X^{(x\oplus x′)}$$.



The cost of this multiplication is roughly $3n$ arithmetic operations. Furthemore, according to this product definition (recall that we do not have phases in this group), we have

$$PP′ = P′P$$


<div class="alert alert-block alert-info">
<b>Ref:</b> Jeroen Dehaene and Bart De Moor, "Clifford group, stabilizer states, and linear and quadratic operations over GF(2)", Phys. Rev. A 68, 042318 (2003).
</div>

In Qiskit there are three ways we can instantiate a Pauli operator:
- labels, e.g.  `Pauli(label='IXZY')` (tensor order is $Q_n\otimes...\otimes Q_1\otimes Q_0$)
- bool vectors `Pauli([True, True, False, False], [True, False, True, False])` (vector order $[q_1, q_2,...,q_n]$
- 0,1 vectors `Pauli([1, 1, 0, 0], [1, 0, 1, 0])` (vector order $[q_1, q_2,...,q_n]$

In [20]:
a = Pauli(label='IXZY')
b = Pauli([True, True, False, False], [True, False, True, False])
c = Pauli([1, 1, 0, 0], [1, 0, 1, 0])
c

Pauli(z=[True, True, False, False], x=[True, False, True, False])

Let us verify that all these representations are the same

In [24]:
a==b

True

In [25]:
a==c

True

`print(pauli)` will return the label representation 

In [26]:
print(a)

IXZY


The number of qubits can be obtained using `pauli.a`

In [27]:
a.numberofqubits

4

### Single qubit Pauli Group
In this representation all the single qubit Pauli operators are defined 

$$P_{00} = Z^0 X^0 = I$$
$$P_{01} = Z^0 X^1 = X$$
$$P_{10} = Z^1 X^0 = Z$$
$$P_{11} = -iZ^1 X^1  = (-i) iY = Y$$

In [28]:
I = Pauli([0], [0])
X = Pauli(label='X')
Z = Pauli([True], [False])
Y = Pauli([1], [1])

Below are four ways we can check the representation of a Pauli 

In [29]:
I

Pauli(z=[False], x=[False])

In [30]:
print(X)

X


In [31]:
Y.to_label()

'Y'

In [12]:
str(Z)

'Z'

These representations can be converted into matrices or sparse matrices

In [32]:
X.to_matrix()

array([[0, 1],
       [1, 0]], dtype=int64)

In [33]:
Z.to_spmatrix()

<2x2 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

Let us test that multiplication of two Paulis X and Z gives indeed the expected result

In [37]:
ytest = X*Z
print(ytest == Y)
ytest = Z*X
print(ytest == Y)

True
True


To sample a random Pauli from the Pauli group one can use

In [39]:
Pauli.random(1)

Pauli(z=[False], x=[True])

To obtain a list of all the elements of the Pauli group one can use

In [17]:
pauli_group(1)

[Pauli(z=[False], x=[False]),
 Pauli(z=[False], x=[True]),
 Pauli(z=[True], x=[True]),
 Pauli(z=[True], x=[False])]

### Two qubit Pauli Group



Recall that the tensor order is $Q_2\otimes Q_1$. For example here we consider the pauli with qubit 0 as $X$ and qubit 1 as $I$.

In [40]:
IX = Pauli(label='IX')

In [41]:
IX.to_matrix()

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int64)

In [42]:
np.kron(np.eye(2),X.to_matrix())

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

The vector order is $x=[q_1, q_2]$ and $z=[q_1, q_2]$

In [43]:
Pauli([0,0],[1,0]).to_matrix()

array([[0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int64)

The `pauli_group` supports two different orderings: 

- `case='weight'` is ordered by Pauli weights and is default
- `case='tensor'` is ordered by I,X,Y,Z counting first qubit fastest

In [44]:
p2 = pauli_group(2)
print([x.to_label() for x in p2])

['II', 'IX', 'IY', 'IZ', 'XI', 'YI', 'ZI', 'XX', 'XY', 'XZ', 'YX', 'YY', 'YZ', 'ZX', 'ZY', 'ZZ']


In [45]:
p2 = pauli_group(2, case='tensor')
print([x.to_label() for x in p2])

['II', 'IX', 'IY', 'IZ', 'XI', 'XX', 'XY', 'XZ', 'YI', 'YX', 'YY', 'YZ', 'ZI', 'ZX', 'ZY', 'ZZ']


## Cliffords

to be added later